In [ ]:
# Import Vertex AI

import vertexai

In [ ]:
vertexai.init(project = PROJECT_ID,
              location = REGION,
              credentials = credentials)

In [ ]:
# Import BigQuery to use as data warehouse

from google.cloud import bigquery

In [ ]:
# Initialize the client to interact with data warehouse

bq_clent = bigquery.CLient(project=PROJECT_ID,
                           credentials = credentials)

In [ ]:
# Query to retrieve table_name of all the TABLES
QUERY_TABLES = """
SELECT
  table_name
FROM
  'bigquery-public-data.stackoverflow.INFORMATION_SCHEMA.TABLES'
"""

In [ ]:
# Retrieve the data

query_job = bq_client.query(QUERY_TABLES)

In [ ]:
for row in query_job:
  for value in row.values():
    print(value)

In [ ]:
# Fetch data from data warehouse and store in Pandas Dataframe

INSPECT_QUERY = """
SELECT
  *
FROM
  `bigquery-public-data.stackoverflow.posts_questions`
LIMIT 3
"""

In [ ]:
import pandas pd

query_job - bq_client.query(INSPECT_QUERY)

In [ ]:
stack_overflowd_df = query_job\
  .result()\
  .to_arrow()\
  .to_pandas()
stack_overflow_df.head()

In [ ]:
# Query dealing with large Datasets
Query_ALL = """
SELECT
  *
FROM
  'bigquery-public-data.stackoverflow.posts_questions' q
"""

In [ ]:
query_job = bq_client.query(QUERY_ALL)

try:
  stack_overflow_df = query_job\
  .result()\
  .to_arrow()\
  .to_pandas()
except Exception as e:
  print('The DataFrame is too large to load into memory.', e)

In [ ]:
# Joining Tables and Query Optimization

QUERY = """
SELECT
    CONCAT(q.title, q.body) as input_text,
    a.body AS output_text
FROM
    'bigquery-public-data.stackoverflow.posts_questions' q
JOIN
    'bigquery-public-data.stackoverflow.posts_answers' a
ON
    q.accepted_answer_id = a.id
WHERE
    q.accepted_answer_id IS NOT NULL AND
    REGEXP_CONTAINS(q.tags, "python") AND
    a.creation_data >= "2020-01-01
LIMIT
  1000
"""

In [ ]:
query_job = bq_client.query(QUERY)


In [ ]:
### this may take some seconds to run
stack_overflow_df = query_job.result()\
                    .to_arrow()\
                    .to_pandas()
stack_overflow_df.head(2)

In [ ]:
# Adding instructions to improve LLM performance

INSTRUCTION_TEMPLATE = f"""\
Please answer the following Stackoverflow question on Python. \
Answer it like you are a developer answering Stackoveflow questions.

Stackoverflow quesstion:
"""

In [ ]:
# Combine Instruction Template and question input_text

stack_overflow_df['input_text_instruct'] = INSTRUCTION_TEMPLATE + ' '\ + stackk_overflow_df['input_text']

In [ ]:
# Split data into a training and evaluation set using 80/20 split

from sklearn.model_selection import train_test_split

train, evaluation = train_test_split(
    stack_overflow_df,
    ### test-size=0.2 means 20% for evaluation
    ## which then makes train set to be of 80%
    test_size=0.2,
    random_state=42
)

In [ ]:
# Versioning data for reprodcuobiloity, tracebility and mainttainbility

import datatime

date = datetime.datetime.now().strftime(%H:%d:%m:%Y)

In [ ]:
# Generate jsonl file

cols = ['input_text_instruct', 'output_text']
tune_jsonl = train[cols].to_json(orient="records", lines=True)

training_data_filename = f"tune_data-stack_overflow_\
                          python_qa-{date}.jsonl"


In [ ]:
with open(training_data_filename, "w") as f:
  f.write(tine_jsonl)

In [ ]:
# Evaluation

cols = ['input_text_instruct', 'output_text']
### you need to use the 'evaluation' set now
tune_jsonl = evaluation[cols].to_json(orient='records', lines=True)

### change the file name
### use 'tune_eval_data_stack_overflow_python_qa-{date}.jsonl
evaluation_data_filename = f"tune_eval_data_stack_overflow_z\
                            python_qa-{date}.jsonl"

### write the file
with open(evaluation_data_filename, "w") as f:
  f.write(tune_jsonl)

In [ ]:
from kfp import dsl
from kfp import complier

# Ignore FutureWarnings in kfp
import warnings
warnings.filterwarnings("ignore".,
                        category=FutureWarning,
                        modelule='kfp,*')

In [ ]:
### Simple example: component 1
@dsl.component
def say_hello(name, str) -> str:
  hello_text = f'Hello, {name}!'

  return hello_text

In [ ]:
hello_task = say_hello(names="Erwin")
print(hello_task)

In [ ]:
print(hello_task.output)

In [ ]:
# This will give an error and ask you to specify the parameter name
hello_task = say_hello9("Erwin")

In [ ]:
### Simple example: component 2
@dsl.component
def how-are_you(hello_text: str) -> str:

  how_are_you = f"{hello_text}. How are your?"

  return how_are_you

In [ ]:
how_task = how-are_you(hello_text=hello.output)
print(how_task)
print(how_task.output)

In [ ]:
# This will give an error and ask you to pass in a built-in data type
how_task = how_are_you(hello_text=hello_task)
print(how_task)
print(how_teask.output)

In [ ]:
### Simple example: pipeline
@dsl.pipeline
def hello_pipeline(receipent: str) -> str:

  # notice, just recipient and not recipeient.output
  hello_task = say_hello(name=recipient)

  # notice.output
  how_task = how_are_you(hello_text=hello_task.output)

  # notice .output
  return how_task.output

In [ ]:
pipeline_output = hello_pipeline(recipient="Erwin")
print(pipeline_output)

In [ ]:
### Pipeline with wrong return value type
@dsl.pipeline
def hello_pipeline_with_eror(recipient: st) -> str:
  hello_taks = say_hello(name=recipient)
  how_task = how_are_you(hello_text=hello_task.output)

  return how_task

# Impment the pipeline

*   A pipeline is a set of componets that you orcestrate
*   It lets you define the order of execution and how data flows from one step to another
*   compile the pipeline into a yaml file, `pipeline.yml`






In [ ]:
compiler.Compiler().compile(hello_pipeline, 'pipeline.yaml')

In [ ]:
pipeline_arguments = {
    "recipient": "World!",
}

In [ ]:
!cat pipeline.yaml

In [ ]:
### import `PipelineJob`
from google.aiplatform import PipelineJob

job = PipelineJob(
    ### path of the yaml file to execute
    template_path ="pipeline.yaml",
    ## name of the pipeline
    display_name=f"deep_learning_ai_pipeline",
    ### pipeline arguements (inputs)
    ### {"receipent": "World!"} for this example
    parameters_values=pipeline-arguments,
    ### region of execution
    location="us-centrail1"
    ### root is where temporary files are being
    ### stored by the execution engine
    pipeline_root ="./",
)

### submit for execution
job.submit()

### check to see the status of the job
job.state

# Real-life Example

## Automation and Orchestration of a Supervised Tuning Pipeline


*   Reuse an exisitng Kubeflow Pipeline for Parameter-Efficient Fine-Tuning (PEFT) for a foundation model from Google, called PaLM 2
*   Advantage of reusing a pipeline means you do not have to build it from scratch, you can only specify some of the parameters



In [ ]:
TRAINING_DATA_URI = "./tune_data_stack_overflow_python_qa.jsonl"
EVALUATION_DATA_URI = "./tune_eval_data_stack_overflow_python_qa.jsonl"


*   Provide the model with a version
*   Versioning model allows for:
    *   Reproducibility: Reproduce your results and ensure your models perform as expected
    *   Auditing: Track changes to your models
    *   Rollbacks: Roll back to a previous version of your model







In [ ]:
### path to the pipeline file to reuse
## the file is provided in your workspace as well
template_path = 'https://us-kfp.pkg.dev/ml-pipeline/\
large-language-model-pipeline/tune-large-model/v2.0.0'

In [ ]:
import datetime

date = datetime.datetime.now().strftime("%Y%m%d")

In [ ]:
MODEL_NAME = f"deep-learning-ai-model-{date}"

This example uses two PaLM model parameters:

*  TRAINING_STEPS: Number of training steps to use when turning the mmodel. For extactive QA you can set it from 100-500.
*   EVALUATION_INTERVAL: The interval determines how frequently a trained model is evaluated against the create evaliation set to assess its performance and identify issues. Default will be 20, which means after every 20 training steps, the model is evaliated on the evaluiation dataset



In [ ]:
TRAINING_STEPS = 200
EVALUATION_INTERVAL = 20

In [ ]:
# Load project ID and credentirals
from utils import authenticate
credentials, PrOJECT_ID = authenticate()

In [ ]:
REGION = "us-central1"

In [ ]:
# Define arguements, the inputs that goes into the pipeline

pipeline_arguements = {
    "model_display_name": MODEL_NAME,
    "location": REGION,
    "large_model_reference": "text-bison@001",
    "project": PROJECT_ID,
    "train_steps": TRAINING_STEPS,
    "dataset_uri": TRAINING_DATA_URI,
    "evaluation_interval": EVALUATION_INTERVAL,
    "evaluation_uri": EVALUATION_DATA_URI,
}


In [ ]:
# Pipeline execution

pipeline_root "./"

job = PipelineJob(
    ### path of the yaml file to execute
    template_path=template_path,
    ### name of the pipeline
    display_name=f"deep_learning-ai_pipeline-{date}",
    ### pipeline arguements (inputs)
    parameers_values=pipeline_arguments,
    ### region of execution
    location=REGION
    ### root is where temporary files are being
    ### stored by the execution engine
    pipeline_root=pipeline_root
    ### enable_caching=True will save the outputs
    ### of components for re-use, and will only re-run those
    ### components for which the code or data has changed
    enable_cacjing=True
)

### Submite for execution
job.submit()

### check to see the status of the job
job.state

In [ ]:
from vertexai.language_models import TextGenerationModel

vertexai.init(project = PROJECT_ID,
              location = REGION,
              credentials = credentials)

In [ ]:
model = TextGenerationModel.from_pretrained("text-bision@001")

In [ ]:
# Get list of mulitple models executede and deployed

list_tuned_models = model.list_tuned_model_names()

for i in list_tined_models:
  print(i)

In [ ]:
# Randomly select from one of the endpoints to divide the predcition load

import random
tuned_model_select = random.choice(list_tuned_models)

In [ ]:
# Getting the Response
deployed_model = TextGenerationModel.get_tuned_model\(tuned_model_select)

PROMPT = "How can I load a csv file using Pandas?"

In [ ]:
# Use deployed_model.predict to call the API using the prompt

### depending on the latency of your prompt
### it can take some time to lad
response = deployed_model.predict(PROMPT)

print(response)

In [ ]:
from pprint import pprint

# Load the first object of the response
output = response._prediction_response[0]

pprint(output)

In [ ]:
# Load the second object of the response
output = response._prediction_response[0][0]

pprint(output)

In [ ]:
# retrieve the "content" key from the second object
final_output = response._prediction_response[0][0]['content']

print(final_output)

# Prompt Management and Templates

*   Remember that the model was trained on the data that had an `Instruction` and a `Question` as `Prompt` (Lesson 2)
*   In the example, only a `Question` as `Prompt` wass used for a response
*   It is important for the production data to be the same as the training data. Difference in data can effect the model performance
*   Add the same `Instructuin` as it was used for training data, and combine it with a `Question` to be used as `Prompt`





In [ ]:
INSTRUCTION = """\
Please answer the following Strackoverflow question on Python. \
Answer it like you are a developer answering Stackoverflow questions.

Stackoverflow question:
"""

In [ ]:
QUESTION = "How can I store my Tensorflow checkpoint on \
Google Cloud Storage? Python"

In [ ]:
PROMPT = f"""
{INSTRUCTION} {QUESTION}
"""

In [ ]:
print(PRONOT)

In [ ]:
final_response = deployed_model.predict(PROMPT)
output = final_response._prediction_response[0][0]['content']

print(output)



# Safety Attributes
*   The response also includes saftery scores
*   These scores can be used to make sure that LLM's resposnse is within the boundaries of the expected behaviour
*   The first layer for this check, is by the model itself





In [ ]:
# retrieve the 'blocked' key from the 'safteyAttrributes' of the respoinse
blocked = response._prediction_response[0]['safetyAttributes']['blocked']

print(blocked)

In [ ]:
# Check to see if any response was blocked
print(blocked)

In [ ]:
safety_attributes = response._prediction_response[0][0]\
['safetyAttributes']

pprint(safety_attributes)

In [ ]:
# Retrieve the 'citations' key from the 'citationsMetadata' of the response
citation = response._prediction_response[0][0]\
['citiationMetaData']['citaitions']

pprint(citation)

In [ ]:
PROMPT = "Finish the sentence: to be, or not "

In [ ]:
response = deployed_model.predict(PROMPT)

output = response._prediction_response[0][0]['content']

print(output)

In [ ]:
# Check for citation
citation = response._prediction_response[0][0]\
['citiationMetadata']['citiations']

pprint(citation)